<a href="https://colab.research.google.com/github/BokyungQQ/davethediver_analysis/blob/main/dave%20the%20diver%20STEAM_%EB%A6%AC%EB%B7%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 리뷰 전처리
- 한글이 아닌 글자 제거
- 작은따옴표 제거
- 연속된 공백 제거
- 좌우 공백 제거
- 최대 글자 255자

In [ ]:
import requests
import time
from datetime import datetime
import pandas as pd
import re
import numpy as np

In [ ]:
dave_the_diver = pd.read_csv('/content/steam_reviews.csv')

In [ ]:
# 날짜 변환
dave_the_diver['formatted_timestamp_updated'] = dave_the_diver['timestamp_updated'].apply(
    lambda timestamp: datetime.utcfromtimestamp(timestamp).strftime('%Y년 %m월 %d일 %H시 %M분 %S초')
)

# 결과 확인
dave_the_diver['formatted_timestamp_updated']

0       2022년 11월 08일 09시 03분 41초
1       2022년 11월 17일 13시 02분 14초
2       2023년 06월 29일 00시 01분 05초
3       2023년 03월 21일 10시 46분 10초
4       2022년 11월 01일 16시 27분 04초
                  ...            
8495    2022년 10월 27일 11시 23분 47초
8496    2022년 10월 27일 10시 44분 17초
8497    2023년 07월 30일 00시 21분 57초
8498    2022년 10월 27일 08시 18분 46초
8499    2022년 10월 27일 08시 10분 27초
Name: formatted_timestamp_updated, Length: 8500, dtype: object

In [ ]:
# 정규 버전 출시일 전날까지로 날짜 제한
# written_during_early_access가 true인 걸로 제한

filtered_data = dave_the_diver[dave_the_diver['written_during_early_access'] == True ]
filtered_data

,Unnamed: 0,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location,timestamp_dev_responded,developer_response,formatted_timestamp_updated
0,0,125143912,"{'steamid': '76561198360804637', 'num_games_ow...",koreana,엔씨보다 너네가 훨 낫다,1667898221,1667898221,True,728,69,0.969466,19,True,False,True,True,NaN,NaN,NaN,2022년 11월 08일 09시 03분 41초
1,1,125625084,"{'steamid': '76561198336084717', 'num_games_ow...",koreana,너희들은 진짜......할수 있으면서 왜!!!!!!!! 할수 있잖아!!!!!!!!,1668690134,1668690134,True,444,93,0.967672,0,True,False,True,True,NaN,NaN,NaN,2022년 11월 17일 13시 02분 14초
3,3,135088278,"{'steamid': '76561198398090507', 'num_games_ow...",koreana,어느덧 30대 후반을 바라보는 직장인은 이미 어떤일에도 의욕을 잃었습니다.\n\n아...,1679395570,1679395570,True,439,12,0.955245,18,True,False,True,True,NaN,1.679403e+09,감사합니다. 너무 좋은 글에 댓글을 안달 수가 없네요...\n고단한 삶에 조금이나마...,2023년 03월 21일 10시 46분 10초
4,4,124782341,"{'steamid': '76561198097817539', 'num_games_ow...",koreana,"넥슨 개발자가 족쇄를 풀고 게임을 만들면, 뭐가 나오는지를 보여줌.",1667320024,1667320024,True,369,23,0.952934,0,False,False,True,True,NaN,NaN,NaN,2022년 11월 01일 16시 27분 04초
5,5,125165870,"{'steamid': '76561198102562894', 'num_games_ow...",koreana,[h1] [b] 데이브 더 다이버 (DAVE THE DIVER) 평가 [/b] [/...,1667934295,1700619736,True,432,16,0.948743,8,False,True,True,True,NaN,NaN,NaN,2023년 11월 22일 02시 22분 16초
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8495,8495,124463117,"{'steamid': '76561198158661776', 'num_games_ow...",koreana,신박하고 잼남,1666869827,1666869827,True,2,0,0.000000,0,True,False,True,True,NaN,NaN,NaN,2022년 10월 27일 11시 23분 47초
8496,8496,124461646,"{'steamid': '76561198822197582', 'num_games_ow...",koreana,저만 그럴까요 게임로딩도 엄청 느리고\n\n 로고 뜨고 게임 진행이 안됩니다 블랙화...,1666867342,1666867457,True,0,0,0.000000,0,True,False,True,True,NaN,NaN,NaN,2022년 10월 27일 10시 44분 17초
8497,8497,124460542,"{'steamid': '76561199032791202', 'num_games_ow...",koreana,내로남불 npc에 여러가지 패러디도 있고 간단하면서도 재밌어요,1666865550,1690676517,True,1,0,0.000000,0,True,False,True,True,NaN,NaN,NaN,2023년 07월 30일 00시 21분 57초
8498,8498,124457367,"{'steamid': '76561198408868655', 'num_games_ow...",koreana,잠깐 해봤는대 재밌네영\n\n물고기 잡는것도 재밌고 초밥 배달하는것도 재밌습니다.\n\n,1666858726,1666858726,True,1,0,0.000000,0,True,False,True,True,NaN,NaN,NaN,2022년 10월 27일 08시 18분 46초


In [ ]:
# 필요한 행만 사용
selected_columns = ['recommendationid', 'author', 'language', 'review','formatted_timestamp_updated', 'voted_up', 'votes_up', 'votes_funny', 'weighted_vote_score', 'comment_count', 'steam_purchase', 'received_for_free', 'written_during_early_access']
selected_dave_the_diver = filtered_data[selected_columns]
filtered_data

,Unnamed: 0,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location,timestamp_dev_responded,developer_response,formatted_timestamp_updated
0,0,125143912,"{'steamid': '76561198360804637', 'num_games_ow...",koreana,엔씨보다 너네가 훨 낫다,1667898221,1667898221,True,728,69,0.969466,19,True,False,True,True,NaN,NaN,NaN,2022년 11월 08일 09시 03분 41초
1,1,125625084,"{'steamid': '76561198336084717', 'num_games_ow...",koreana,너희들은 진짜......할수 있으면서 왜!!!!!!!! 할수 있잖아!!!!!!!!,1668690134,1668690134,True,444,93,0.967672,0,True,False,True,True,NaN,NaN,NaN,2022년 11월 17일 13시 02분 14초
3,3,135088278,"{'steamid': '76561198398090507', 'num_games_ow...",koreana,어느덧 30대 후반을 바라보는 직장인은 이미 어떤일에도 의욕을 잃었습니다.\n\n아...,1679395570,1679395570,True,439,12,0.955245,18,True,False,True,True,NaN,1.679403e+09,감사합니다. 너무 좋은 글에 댓글을 안달 수가 없네요...\n고단한 삶에 조금이나마...,2023년 03월 21일 10시 46분 10초
4,4,124782341,"{'steamid': '76561198097817539', 'num_games_ow...",koreana,"넥슨 개발자가 족쇄를 풀고 게임을 만들면, 뭐가 나오는지를 보여줌.",1667320024,1667320024,True,369,23,0.952934,0,False,False,True,True,NaN,NaN,NaN,2022년 11월 01일 16시 27분 04초
5,5,125165870,"{'steamid': '76561198102562894', 'num_games_ow...",koreana,[h1] [b] 데이브 더 다이버 (DAVE THE DIVER) 평가 [/b] [/...,1667934295,1700619736,True,432,16,0.948743,8,False,True,True,True,NaN,NaN,NaN,2023년 11월 22일 02시 22분 16초
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8495,8495,124463117,"{'steamid': '76561198158661776', 'num_games_ow...",koreana,신박하고 잼남,1666869827,1666869827,True,2,0,0.000000,0,True,False,True,True,NaN,NaN,NaN,2022년 10월 27일 11시 23분 47초
8496,8496,124461646,"{'steamid': '76561198822197582', 'num_games_ow...",koreana,저만 그럴까요 게임로딩도 엄청 느리고\n\n 로고 뜨고 게임 진행이 안됩니다 블랙화...,1666867342,1666867457,True,0,0,0.000000,0,True,False,True,True,NaN,NaN,NaN,2022년 10월 27일 10시 44분 17초
8497,8497,124460542,"{'steamid': '76561199032791202', 'num_games_ow...",koreana,내로남불 npc에 여러가지 패러디도 있고 간단하면서도 재밌어요,1666865550,1690676517,True,1,0,0.000000,0,True,False,True,True,NaN,NaN,NaN,2023년 07월 30일 00시 21분 57초
8498,8498,124457367,"{'steamid': '76561198408868655', 'num_games_ow...",koreana,잠깐 해봤는대 재밌네영\n\n물고기 잡는것도 재밌고 초밥 배달하는것도 재밌습니다.\n\n,1666858726,1666858726,True,1,0,0.000000,0,True,False,True,True,NaN,NaN,NaN,2022년 10월 27일 08시 18분 46초


In [ ]:
# 리뷰 전처리
filtered_data['review'] = filtered_data['review'] \
  .replace(r'[^가-힣 ]', ' ', regex=True) \
  .replace("'", '') \
  .replace(r'\s+', ' ', regex=True) \
  .str.strip() \
  .str[:255]

filtered_data.head()

<ipython-input-11-ce0181f6f7e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['review'] = filtered_data['review'] \


,Unnamed: 0,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location,timestamp_dev_responded,developer_response,formatted_timestamp_updated
0,0,125143912,"{'steamid': '76561198360804637', 'num_games_ow...",koreana,엔씨보다 너네가 훨 낫다,1667898221,1667898221,True,728,69,0.969466,19,True,False,True,True,NaN,NaN,NaN,2022년 11월 08일 09시 03분 41초
1,1,125625084,"{'steamid': '76561198336084717', 'num_games_ow...",koreana,너희들은 진짜 할수 있으면서 왜 할수 있잖아,1668690134,1668690134,True,444,93,0.967672,0,True,False,True,True,NaN,NaN,NaN,2022년 11월 17일 13시 02분 14초
3,3,135088278,"{'steamid': '76561198398090507', 'num_games_ow...",koreana,어느덧 대 후반을 바라보는 직장인은 이미 어떤일에도 의욕을 잃었습니다 아마 제 또래...,1679395570,1679395570,True,439,12,0.955245,18,True,False,True,True,NaN,1.679403e+09,감사합니다. 너무 좋은 글에 댓글을 안달 수가 없네요...\n고단한 삶에 조금이나마...,2023년 03월 21일 10시 46분 10초
4,4,124782341,"{'steamid': '76561198097817539', 'num_games_ow...",koreana,넥슨 개발자가 족쇄를 풀고 게임을 만들면 뭐가 나오는지를 보여줌,1667320024,1667320024,True,369,23,0.952934,0,False,False,True,True,NaN,NaN,NaN,2022년 11월 01일 16시 27분 04초
5,5,125165870,"{'steamid': '76561198102562894', 'num_games_ow...",koreana,데이브 더 다이버 평가 가성비 제값 줘도 안 아까움 세일도 살 만함 세일까지 존버 ...,1667934295,1700619736,True,432,16,0.948743,8,False,True,True,True,NaN,NaN,NaN,2023년 11월 22일 02시 22분 16초


In [ ]:
filtered_data['review']

0                                           엔씨보다 너네가 훨 낫다
1                                너희들은 진짜 할수 있으면서 왜 할수 있잖아
3       어느덧 대 후반을 바라보는 직장인은 이미 어떤일에도 의욕을 잃었습니다 아마 제 또래...
4                     넥슨 개발자가 족쇄를 풀고 게임을 만들면 뭐가 나오는지를 보여줌
5       데이브 더 다이버 평가 가성비 제값 줘도 안 아까움 세일도 살 만함 세일까지 존버 ...
                              ...                        
8495                                              신박하고 잼남
8496    저만 그럴까요 게임로딩도 엄청 느리고 로고 뜨고 게임 진행이 안됩니다 블랙화면 그대로에요
8497                      내로남불 에 여러가지 패러디도 있고 간단하면서도 재밌어요
8498            잠깐 해봤는대 재밌네영 물고기 잡는것도 재밌고 초밥 배달하는것도 재밌습니다
8499                작살 발사할때 마우스 오른쪽 클릭 해도 안먹힐때가 많은데 나만그런가
Name: review, Length: 3560, dtype: object

- 아무것도 안 써있는 리뷰 제거

In [ ]:
# review 열에서 null인 행 제거
filtered_data = filtered_data.dropna(subset=['review'])
# 앞뒤 공백을 제거하고 빈 문자열 확인 후 제거
filtered_data = filtered_data[filtered_data['review'].str.strip().astype(bool)]
len(filtered_data)
# 분석할 리뷰 개수 : 3416개

3416

In [ ]:
# filtered_data = pd.read_csv('/content/filtered_data_1.csv')

# 전체 리뷰 추천 상위 10개 확인

In [ ]:
# 전체 리뷰 중에서 추천 상위 10개 항목 확인
filtered_data_top10 = filtered_data.sort_values(by='votes_up',ascending=False).head(10)
select = ['review','votes_up']
filtered_data_top10[select]
# 게임에 대한 긍정적 반응 : | 728+444+369+287+247+225
# 기타 : 1 | 439
# 게임에 대한 부정적 반응 : | 432
# 멀티 지원 요청 : 1 | 288

,review,votes_up
0,엔씨보다 너네가 훨 낫다,728
1,너희들은 진짜 할수 있으면서 왜 할수 있잖아,444
3,어느덧 대 후반을 바라보는 직장인은 이미 어떤일에도 의욕을 잃었습니다 아마 제 또래...,439
5,데이브 더 다이버 평가 가성비 제값 줘도 안 아까움 세일도 살 만함 세일까지 존버 ...,432
4,넥슨 개발자가 족쇄를 풀고 게임을 만들면 뭐가 나오는지를 보여줌,369
12,차는 너네가 따라 마셔 십 세들아,297
13,초반인데 재미있네요 와이프랑도 할수 있게 멀티 지원 부탁드립니다 와이프는 식당에서 ...,288
17,수많은 실패를 겪고도 다시한번 새로운 도전에 발을 내딛는 넥슨에게 찬사를 많은 분들...,287
41,오늘 정식 출시 아니야 왜 안 내 빨리 내줘 오늘 정식 출시 아니야 왜 안 내 빨리...,247
16,빨리일해민트로켓밥도먹지마잠도자지마회사에서퇴근도하지마빨리일해민트로켓감금된채로일만해서빨...,225


# 리뷰 형태소 분석

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 46.8 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt # 형태소 분석기 Okt
from collections import Counter
from konlpy.tag import Komoran

In [ ]:
stop_words = ['진짜','아주','정도','이것','저것','가성','완전','정말','1','2','3','4','5','6','이거','지금','이','\n','게',"?","에","가","는","들","도","거","대","임","안","은","??","한","-","다","함","m","넥슨","시간",
              '겜', '더', '제발', '것', '수', '때', '좀','왜','그', '내','의','를','로','을','너무','게임','으로','빨리','시','플레이']

In [ ]:
# 명사만 추출
tokenizer_noun = Okt()
filtered_data['리뷰 텍스트 분석 명사'] = filtered_data['review'].apply(lambda x: tokenizer_noun.nouns(x))

In [ ]:
# 명사가 아니라 형태소 형태로 존재하는 단어들도 확인하기 위해 형태소 추출
tokenizer = Okt()
filtered_data['리뷰 텍스트 분석 형태소'] = filtered_data['review'].apply(tokenizer.morphs)
filtered_data['리뷰 텍스트 분석 형태소']

0                                [엔씨, 보다, 너, 네, 가, 훨, 낫다]
1                    [너희, 들, 은, 진짜, 할수, 있으면서, 왜, 할수, 있잖아]
2       [어느덧, 대, 후반, 을, 바라보는, 직장인, 은, 이미, 어떤, 일, 에도, 의...
3       [넥슨, 개발자, 가, 족쇄, 를, 풀, 고, 게임, 을, 만들면, 뭐, 가, 나오...
4       [데이브, 더, 다이버, 평가, 가성, 비, 제, 값, 줘도, 안, 아까, 움, 세...
                              ...                        
3411                                       [신, 박하, 고, 잼남]
3412    [저만, 그럴까, 요, 게임, 로딩, 도, 엄청, 느리고, 로고, 뜨고, 게임, 진...
3413          [내, 로남불, 에, 여러가지, 패러디, 도, 있고, 간단하면서도, 재밌어요]
3414    [잠깐, 해봤는대, 재밌네, 영, 물고기, 잡는것도, 재밌고, 초밥, 배달, 하는것...
3415    [작살, 발사, 할, 때, 마우스, 오른쪽, 클릭, 해도, 안, 먹힐, 때, 가, ...
Name: 리뷰 텍스트 분석 형태소, Length: 3416, dtype: object

In [ ]:
# 명사 키워드 분석
false_filtered_words = np.hstack(filtered_data['리뷰 텍스트 분석 명사'].values)

# 단어의 빈도수를 계산
false_word_count = Counter(false_filtered_words)
false_word_count = Counter({x : false_word_count[x] for x in false_word_count if x not in stop_words})
false_word_count = sorted(false_word_count.items(), key=lambda x: x[1], reverse=True)

# 100번 이상 언급된 키워드만 출력
false_word_count = {word: count for word, count in false_word_count if count >= 100}

# 13개의 키워드로 축약
false_word_count

{'정식': 762,
 '출시': 546,
 '컨텐츠': 329,
 '초밥': 242,
 '물고기': 232,
 '재미': 217,
 '스토리': 201,
 '기대': 177,
 '생각': 175,
 '힐링': 159,
 '도트': 153,
 '바다': 138,
 '집': 129,
 '느낌': 123,
 '그래픽': 122,
 '요소': 122,
 '스시': 117,
 '계속': 110,
 '돈': 106,
 '타이': 104,
 '운영': 102,
 '그냥': 101,
 '마을': 100}

In [ ]:
# 추출한 형태소 키워드 분석 진행
false_filtered_words_mor = np.hstack(filtered_data['리뷰 텍스트 분석 형태소'].values)

# 단어의 빈도수를 계산
false_word_count_mor = Counter(false_filtered_words_mor)
false_word_count_mor = Counter({x : false_word_count_mor[x] for x in false_word_count_mor \
                               if x not in stop_words})
false_word_count_mor = sorted(false_word_count_mor.items(), key=lambda x: x[1], reverse=True)

# 100번 이상 언급된 키워드만 출력
false_word_count_mor = {word: count for word, count in false_word_count_mor if count >= 100}
false_word_count_mor

{'정식': 762,
 '출시': 546,
 '컨텐츠': 329,
 '하는': 315,
 '할': 314,
 '에서': 312,
 '하고': 301,
 '만': 290,
 '초밥': 242,
 '적': 240,
 '잘': 234,
 '물고기': 232,
 '갓': 229,
 '재미': 217,
 '스토리': 201,
 '얼리': 187,
 '까지': 186,
 '나': 183,
 '입니다': 182,
 '기대': 177,
 '생각': 175,
 '과': 173,
 '힐링': 159,
 '아직': 154,
 '도트': 153,
 '이런': 151,
 '재밌음': 148,
 '인': 148,
 '있는': 143,
 '와': 138,
 '바다': 138,
 '집': 129,
 '되는': 129,
 '느낌': 123,
 '합니다': 123,
 '그래픽': 122,
 '요소': 122,
 '많이': 120,
 '고': 117,
 '스시': 117,
 '못': 111,
 '하면': 111,
 '요': 111,
 '계속': 110,
 '어인': 109,
 '제': 108,
 '해서': 108,
 '중': 107,
 '그냥': 106,
 '돈': 106,
 '재밌어요': 105,
 '타이': 104,
 '운영': 102,
 '재밌다': 102,
 '해': 101,
 '마을': 100,
 '에는': 100,
 '이렇게': 100}

In [ ]:
# 명사 분석 결과와 형태소 분석 결과의 공통 단어 추출
common_words = set(false_word_count_mor.keys()) & set(false_word_count.keys())

print("공통 단어:", common_words)

공통 단어: {'바다', '마을', '초밥', '그래픽', '집', '생각', '힐링', '도트', '타이', '요소', '컨텐츠', '계속', '스토리', '재미', '기대', '운영', '정식', '느낌', '돈', '스시', '출시', '그냥', '물고기'}


명사 분석 결과와 형태소 분석 결과가 모두 일치하므로 명사 분석 결과 중심으로 다시 세부 분석 진행

# 세부 의견 확인

키워드 빈도 1,2위를 차지한 '정식','출시'의 경우 세부 의견을 확인하지 않아도 dave the diver의 정식 출시를 기대하는 내용으로 예상

따라서 세부의견을 확인하지 않음

## 공동 3위 : 컨텐츠 ( 359회 )

In [ ]:
# 컨텐츠, 콘텐츠 둘 다 같은 의미를 가지므로 2개의 단어 중 하나라도 들어가는 리뷰로 추림
opinion_contents = [(review,voted_up,votes_up) for review,voted_up,votes_up in zip(filtered_data['review'],filtered_data['voted_up'],filtered_data['votes_up']) if '컨텐츠' in review or '콘텐츠' in review]
opinion_contents = pd.DataFrame.from_dict(opinion_contents)
opinion_contents

,0,1,2
0,빨리일해민트로켓밥도먹지마잠도자지마회사에서퇴근도하지마빨리일해민트로켓감금된채로일만해서빨...,True,225
1,치킨을 한 번 참을만한 가치가 있는 게임 평가 상단은 구매를 고려하는 분들을 위해 ...,True,102
2,다양한 시도를 한 넥슨 드디어 일냈다 꽤 뛰어난 도트퀄리티 자연스러운 연출 적당한 ...,True,99
3,이 게임의 단점은 아직도 얼리엑세스라는거임 빨리 완성된 컨텐츠를 가져오지 않으면 산...,True,29
4,이 게임은 그냥 향후 가게 분점으로 다른 해양과 다른 물고기만 늘어놔도 컨텐츠가 늘...,True,28
...,...,...,...
299,즐길 수 있는 콘텐츠가 정말 많은 것 같음 즐겁게 플레이 했고 정식 발매가 기다려지...,True,0
300,진짜 기대안했는데 갓겜이다 이대로 컨텐츠만 꾸준히 나와다오,True,0
301,피처폰으로 타이쿤게임하던 그감성 오랜만에 느꼈음 굉장히 재미있음 근데 얼액감안하더라...,True,0
302,얼엑 시간 후기 난 이 게임 내년 도 가능하다고 봄 넥슨인데 인디장르로 장르 불문하...,True,3


In [ ]:
# 총 추천 구하기
sum(opinion_contents[2])

741

### 긍정적 리뷰

In [ ]:
# 긍정적 리뷰만 추림 (288개)
opinion_contents_plus = opinion_contents[opinion_contents[1] == True]
opinion_contents_plus

,0,1,2
0,빨리일해민트로켓밥도먹지마잠도자지마회사에서퇴근도하지마빨리일해민트로켓감금된채로일만해서빨...,True,225
1,치킨을 한 번 참을만한 가치가 있는 게임 평가 상단은 구매를 고려하는 분들을 위해 ...,True,102
2,다양한 시도를 한 넥슨 드디어 일냈다 꽤 뛰어난 도트퀄리티 자연스러운 연출 적당한 ...,True,99
3,이 게임의 단점은 아직도 얼리엑세스라는거임 빨리 완성된 컨텐츠를 가져오지 않으면 산...,True,29
4,이 게임은 그냥 향후 가게 분점으로 다른 해양과 다른 물고기만 늘어놔도 컨텐츠가 늘...,True,28
...,...,...,...
299,즐길 수 있는 콘텐츠가 정말 많은 것 같음 즐겁게 플레이 했고 정식 발매가 기다려지...,True,0
300,진짜 기대안했는데 갓겜이다 이대로 컨텐츠만 꾸준히 나와다오,True,0
301,피처폰으로 타이쿤게임하던 그감성 오랜만에 느꼈음 굉장히 재미있음 근데 얼액감안하더라...,True,0
302,얼엑 시간 후기 난 이 게임 내년 도 가능하다고 봄 넥슨인데 인디장르로 장르 불문하...,True,3


In [ ]:
# 키워드 기준으로 어구 추출
def remove_before_content(text):
    matches = re.finditer(r'[콘컨]텐츠', text) # finditer 이용해서 여러 개의 어구 추출
    results = []
    for match in matches:
        start_index = match.start() - 5  #  키워드 앞에서 5글자부터 추출
        end_index = match.end() + 15      # 키워드 뒤에서 15글자까지 추출
        if start_index < 0:
            start_index = 0  # 시작 인덱스가 음수일 경우 0으로 설정
        if end_index > len(text):
            end_index = len(text)  # 종료 인덱스가 문자열 길이보다 큰 경우 문자열 끝까지 설정
        result = text[start_index:end_index]
        results.append(result)
    return results

# 결과를 새로운 열에 저장
opinion_contents_plus['긍정'] = opinion_contents_plus[0].apply(remove_before_content)
opinion_contents_plus

<ipython-input-67-c68ce90c42d2>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_contents_plus['긍정'] = opinion_contents_plus[0].apply(remove_before_content)


,0,1,2,긍정
0,빨리일해민트로켓밥도먹지마잠도자지마회사에서퇴근도하지마빨리일해민트로켓감금된채로일만해서빨...,True,225,[참겠어빨리컨텐츠추가도하고정발도하고일해빨리]
1,치킨을 한 번 참을만한 가치가 있는 게임 평가 상단은 구매를 고려하는 분들을 위해 ...,True,102,[볼륨 및 컨텐츠 어렵지 않은 플레이 방식 ]
2,다양한 시도를 한 넥슨 드디어 일냈다 꽤 뛰어난 도트퀄리티 자연스러운 연출 적당한 ...,True,99,"[꽤 즐길 컨텐츠가 꽤 많다 현재 양식장을 , 더 많은 컨텐츠가 등장할 것으로 보인..."
3,이 게임의 단점은 아직도 얼리엑세스라는거임 빨리 완성된 컨텐츠를 가져오지 않으면 산...,True,29,[ 완성된 컨텐츠를 가져오지 않으면 산소부족]
4,이 게임은 그냥 향후 가게 분점으로 다른 해양과 다른 물고기만 늘어놔도 컨텐츠가 늘...,True,28,"[늘어놔도 컨텐츠가 늘것이고 스토리상으로는 , 도 게임 컨텐츠는 마르지 않을 것이다]"
...,...,...,...,...
299,즐길 수 있는 콘텐츠가 정말 많은 것 같음 즐겁게 플레이 했고 정식 발매가 기다려지...,True,0,[수 있는 콘텐츠가 정말 많은 것 같음 즐겁]
300,진짜 기대안했는데 갓겜이다 이대로 컨텐츠만 꾸준히 나와다오,True,0,[ 이대로 컨텐츠만 꾸준히 나와다오]
301,피처폰으로 타이쿤게임하던 그감성 오랜만에 느꼈음 굉장히 재미있음 근데 얼액감안하더라...,True,0,[간격이나 컨텐츠해금 같은것들을 조금 느린템]
302,얼엑 시간 후기 난 이 게임 내년 도 가능하다고 봄 넥슨인데 인디장르로 장르 불문하...,True,3,[수 있음 컨텐츠 하나하나의 깊이는 깊지]


In [ ]:
tokenizer = Okt()
# 키워드 분석을 위해 리스트를 문자열로 변환
opinion_contents_plus['긍정'] = opinion_contents_plus['긍정'].apply(lambda x: ', '.join(x))
opinion_contents_plus['리뷰 분석 형태소'] = opinion_contents_plus['긍정'].apply(tokenizer.morphs)
opinion_contents_plus['리뷰 분석 형태소']

<ipython-input-73-29c1a0adbb4d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_contents_plus['긍정'] = opinion_contents_plus['긍정'].apply(lambda x: ', '.join(x))
<ipython-input-73-29c1a0adbb4d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_contents_plus['리뷰 분석 형태소'] = opinion_contents_plus['긍정'].apply(tokenizer.morphs)


0           [참겠어, 빨리, 컨텐츠, 추, 가도, 하고, 정발, 도하, 고, 일해, 빨리]
1                         [볼륨, 및, 컨텐츠, 어렵지, 않은, 플레이, 방식]
2      [꽤, 즐길, 컨텐츠, 가, 꽤, 많다, 현재, 양식, 장, 을, ,, 더, 많은,...
3                     [완성, 된, 컨텐츠, 를, 가져오지, 않으면, 산소, 부족]
4      [늘어놔도, 컨텐츠, 가, 늘것이고, 스토리, 상, 으로는, ,, 도, 게임, 컨텐...
                             ...                        
299                   [수, 있는, 콘텐츠, 가, 정말, 많은, 것, 같음, 즐겁]
300                             [이대로, 컨텐츠, 만, 꾸준히, 나와다오]
301                [간격, 이나, 컨텐츠, 해금, 같은것, 들을, 조금, 느린, 템]
302                   [수, 있음, 컨텐츠, 하나, 하나, 의, 깊이, 는, 깊지]
303                    [수, 있는, 컨텐츠, 도, 최대한, 잘, 짜냈다는, 생각]
Name: 리뷰 분석 형태소, Length: 288, dtype: object

In [ ]:
# 추출한 형태소 키워드 분석 진행
opinion_contents_plus_mor = np.hstack(opinion_contents_plus['리뷰 분석 형태소'].values)

# 단어의 빈도수를 계산
opinion_contents_plus_mor = Counter(opinion_contents_plus_mor)
opinion_contents_plus_mor = Counter({x : opinion_contents_plus_mor[x] for x in opinion_contents_plus_mor if x not in stop_words})
opinion_contents_plus_mor = sorted(opinion_contents_plus_mor.items(), key=lambda x: x[1], reverse=True)

# 10번 이상 언급된 키워드만 출력
opinion_contents_plus_mor = {word: count for word, count in opinion_contents_plus_mor if count >= 10}
opinion_contents_plus_mor

# 키워드 분석 결과 정식 출시 될 내용을 기대하고 있으며 ( 정식, 출시, 추가 )
# 또한 현재 컨텐츠 분량에 대해 만족하는 리뷰가 많음 ( 많고, 다양한, 많은, 많아서 )

{'컨텐츠': 315,
 ',': 52,
 '정식': 30,
 '콘텐츠': 28,
 '추가': 21,
 '출시': 16,
 '고': 14,
 '다양한': 14,
 '적': 14,
 '기대': 14,
 '있는': 13,
 '많고': 13,
 '새로운': 13,
 '되는': 13,
 '즐길': 12,
 '스토리': 11,
 '많아서': 11,
 '반복': 11,
 '하나': 11,
 '만': 11,
 '많은': 10,
 '인': 10}

In [ ]:
# 추천 상위 10개 확인
opinion_contents_plus.sort_values(by=2,ascending=False).head(10)

# 추천 상위 3개가 압도적인 값을 기록
# 컨텐츠 분량과 다양성에 만족한 리뷰 : 2 | 102+99 -> 27%
# 컨텐츠 추가를 기원하는 리뷰 ( 기대 ) : 7 | 225+29+29+29+28+11+10 -> 48%
# 컨텐츠 부족에 대한 리뷰 : 1 | 11 -> 1%

,0,1,2,긍정,리뷰 분석 형태소
0,빨리일해민트로켓밥도먹지마잠도자지마회사에서퇴근도하지마빨리일해민트로켓감금된채로일만해서빨...,True,225,참겠어빨리컨텐츠추가도하고정발도하고일해빨리,"[참겠어, 빨리, 컨텐츠, 추, 가도, 하고, 정발, 도하, 고, 일해, 빨리]"
1,치킨을 한 번 참을만한 가치가 있는 게임 평가 상단은 구매를 고려하는 분들을 위해 ...,True,102,볼륨 및 컨텐츠 어렵지 않은 플레이 방식,"[볼륨, 및, 컨텐츠, 어렵지, 않은, 플레이, 방식]"
2,다양한 시도를 한 넥슨 드디어 일냈다 꽤 뛰어난 도트퀄리티 자연스러운 연출 적당한 ...,True,99,"꽤 즐길 컨텐츠가 꽤 많다 현재 양식장을 , 더 많은 컨텐츠가 등장할 것으로 보인다 낮","[꽤, 즐길, 컨텐츠, 가, 꽤, 많다, 현재, 양식, 장, 을, ,, 더, 많은,..."
3,이 게임의 단점은 아직도 얼리엑세스라는거임 빨리 완성된 컨텐츠를 가져오지 않으면 산...,True,29,완성된 컨텐츠를 가져오지 않으면 산소부족,"[완성, 된, 컨텐츠, 를, 가져오지, 않으면, 산소, 부족]"
5,너무 재미있어요 컨텐츠 거의다해서 정출만 기다리는 중인데 혹시 정식출시되면 원래 세...,True,29,미있어요 컨텐츠 거의다해서 정출만 기다리는,"[미있어요, 컨텐츠, 거의, 다해, 서, 정, 출만, 기다리는]"
6,핫픽스좀 그만하고 도데체 언제 이거 컨텐츠 추가됍니까,True,29,제 이거 컨텐츠 추가됍니까,"[제, 이, 거, 컨텐츠, 추가, 됍니까]"
4,이 게임은 그냥 향후 가게 분점으로 다른 해양과 다른 물고기만 늘어놔도 컨텐츠가 늘...,True,28,"늘어놔도 컨텐츠가 늘것이고 스토리상으로는 , 도 게임 컨텐츠는 마르지 않을 것이다","[늘어놔도, 컨텐츠, 가, 늘것이고, 스토리, 상, 으로는, ,, 도, 게임, 컨텐..."
11,게임 자체는 재밌음 문제는 없데이트 반년가량 컨텐츠 추가가 전혀 없음 핫픽스니 소규...,True,11,반년가량 컨텐츠 추가가 전혀 없음 핫픽스니,"[반년, 가량, 컨텐츠, 추가, 가, 전혀, 없음, 핫픽스, 니]"
8,재미있고 컨텐츠도 많아보이나 실제해보면 플레이타임이 상당히 짧은편,True,11,재미있고 컨텐츠도 많아보이나 실제해보면 플,"[재미있고, 컨텐츠, 도, 많아, 보이, 나, 실제, 해보면, 플]"
7,아 빨리 추가컨텐츠 주세요 현기증 난단 말이에요 아 빨리 추가 내주세요 현기증 난단...,True,10,빨리 추가컨텐츠 주세요 현기증 난단 말이에,"[빨리, 추가, 컨텐츠, 주세요, 현기증, 난단, 말, 이, 에]"


### 부정적 리뷰

In [ ]:
# 부정적 리뷰만 추림 (16개)
opinion_contents_minus = opinion_contents[opinion_contents[1] != True]
opinion_contents_minus

,0,1,2
10,엄청나게 호평이라 궁금해서 사봤는데 게임이 얕다는 생각이 들게 됨 더 보여주고 싶은...,False,12
19,처음 구매하고 시간정도는 잠수장비 업글하고 하면서 재미있게 했는데 얼리라서 소지금 ...,False,7
28,제작진 하고싶은거 다 넣은것같은 게임 미니게임도 많고 컨텐츠도 많지만 아쉬움도 많은...,False,0
56,각 잡고 하면 다섯 시간도 안 나올 플레임타임에 원이라는 가격은 아직 비싸다 원짜리...,False,5
64,초반에 잠깐 잼있고 컨텐츠가 지루해짐,False,3
68,뭔가 프로 부족한 게임 메인 컨텐츠인 물고기 잡기 회떠서 팔기 자체는 재밌는데 후반...,False,3
72,게임이 너무 자주 튕깁니다 그리고 아직 얼리 액세스 단계라 한 열흘 지나고 하면 할...,False,0
75,장점 극 초반 바닷속이라는 배경이 신박하고 초반엔 힐링되는 느낌 적당한 타이쿤 식 ...,False,2
76,시간 가까히 해보니까 느낌점 한 일은 할만합니다 일넘어가면서 컨텐츠 전부 다 해보니...,False,1
77,플탐이 얼리엑세스인거 감안해도 생각보다 너무 짧고 회차 엔딩직전에 너무많이죽어서 리...,False,0


In [ ]:
# 부정 리뷰 키워드 분석 진행 (10개 이상일 경우 진행)
# 결과를 새로운 열에 저장
opinion_contents_minus['부정'] = opinion_contents_minus[0].apply(remove_before_content)
# 부정적 리뷰 키워드 분석 진행
tokenizer = Okt()
opinion_contents_minus['부정'] = opinion_contents_minus['부정'].apply(lambda x: ', '.join(x))
opinion_contents_minus['리뷰 분석 형태소'] = opinion_contents_minus['부정'].apply(tokenizer.morphs)
# 추출한 형태소 키워드 분석 진행
opinion_contents_minus_mor = np.hstack(opinion_contents_minus['리뷰 분석 형태소'].values)
# 단어의 빈도수를 계산
opinion_contents_minus_mor = Counter(opinion_contents_minus_mor)
opinion_contents_minus_mor = Counter({x : opinion_contents_minus_mor[x] for x in opinion_contents_minus_mor if x not in stop_words})
opinion_contents_minus_mor = sorted(opinion_contents_minus_mor.items(), key=lambda x:x[1], reverse=True)
# 10번 이상 언급된 키워드만 출력
opinon_contents_minus_mor = {word : count for word, count in opinion_contents_minus_mor if count>= 2}
opinon_contents_minus_mor
# 특별하게 예상 가능한 단어 등장하지 않음 (많지만의 경우 뒤가 더 중요하므로 예상 불가)

<ipython-input-83-9af96e04c97e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_contents_minus['부정'] = opinion_contents_minus[0].apply(remove_before_content)
<ipython-input-83-9af96e04c97e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_contents_minus['부정'] = opinion_contents_minus['부정'].apply(lambda x: ', '.join(x))
<ipython-input-83-9af96e04c97e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

{'컨텐츠': 17,
 '메인': 4,
 '할': 3,
 ',': 3,
 '많지만': 2,
 '인': 2,
 '하면': 2,
 '주고': 2,
 '많이': 2,
 '하': 2,
 '콘텐츠': 2,
 '못': 2,
 '된': 2}

In [ ]:
# 추천 상위 10개 확인
opinion_contents_minus.sort_values(by=2, ascending=False).head(10)
# 컨텐츠 부족에 대해 불만 : 3 | 12+7+5
# 컨텐츠가 지루하다는 것에 대한 불만 : 6 | 3+2+2+1+1+1
# 기타 : 1 | 3

,0,1,2,부정,리뷰 분석 형태소
10,엄청나게 호평이라 궁금해서 사봤는데 게임이 얕다는 생각이 들게 됨 더 보여주고 싶은...,False,12,"만에 메인컨텐츠가 막혀서 딱히 할 게 없었, 다한정도 컨텐츠가 엄청 많지만 메인인 해양","[만에, 메인, 컨텐츠, 가, 막혀서, 딱히, 할, 게, 없었, ,, 다한, 정도,..."
19,처음 구매하고 시간정도는 잠수장비 업글하고 하면서 재미있게 했는데 얼리라서 소지금 ...,False,7,"반년동안 컨텐츠 업데이트는 거의 없다고 봐, 도 지금 컨텐츠량 생각하면 정가 주고 할","[반년, 동안, 컨텐츠, 업데이트, 는, 거의, 없다고, 봐, ,, 도, 지금, 컨..."
56,각 잡고 하면 다섯 시간도 안 나올 플레임타임에 원이라는 가격은 아직 비싸다 원짜리...,False,5,나오는데 컨텐츠가 많이 빈약하다 경영 게임,"[나오는데, 컨텐츠, 가, 많이, 빈약하다, 경영, 게임]"
64,초반에 잠깐 잼있고 컨텐츠가 지루해짐,False,3,잼있고 컨텐츠가 지루해짐,"[잼있고, 컨텐츠, 가, 지루해, 짐]"
68,뭔가 프로 부족한 게임 메인 컨텐츠인 물고기 잡기 회떠서 팔기 자체는 재밌는데 후반...,False,3,임 메인 컨텐츠인 물고기 잡기 회떠서 팔기,"[임, 메인, 컨텐츠, 인, 물고기, 잡기, 회, 떠서, 팔기]"
75,장점 극 초반 바닷속이라는 배경이 신박하고 초반엔 힐링되는 느낌 적당한 타이쿤 식 ...,False,2,노가다성 컨텐츠로 인한 루즈함 보스몹 잡는,"[노가다, 성, 컨텐츠, 로, 인한, 루즈, 함, 보스, 몹, 잡는]"
79,압도적 긍정적이라길래 해봤는데 년전에 폰으로 하던 게임 수준이다 그당시엔 할게 그런...,False,2,고 온갖 컨텐츠가 포함된 게임을 많이 겪다,"[고, 온갖, 컨텐츠, 가, 포함, 된, 게임, 을, 많이, 겪다]"
76,시간 가까히 해보니까 느낌점 한 일은 할만합니다 일넘어가면서 컨텐츠 전부 다 해보니...,False,1,어가면서 컨텐츠 전부 다 해보니까 할께 없,"[어가, 면서, 컨텐츠, 전부, 다, 해보니까, 할께, 없]"
78,재밌긴한데 얼리액세스라도 원은 좀 쎄다 생각함 게임 가 노가다 폰게임을 바람의나라처...,False,1,"낌 지금 콘텐츠의 배라도 만원을 못 넘을 , 단순하고 콘텐츠가 몇개 없음","[낌, 지금, 콘텐츠, 의, 배, 라도, 만원, 을, 못, 넘을, ,, 단순하고, ..."
80,반년째 제대로된 컨텐츠 추가없는 얼엑겜 이돈주고 사야할 이유가없음,False,1,제대로된 컨텐츠 추가없는 얼엑겜 이돈주고,"[제대로, 된, 컨텐츠, 추가, 없는, 얼, 엑, 겜, 이돈, 주고]"


## 공동 3위 : 초밥 / 스시 ( 359회 )

In [ ]:
# '초밥'과 '스시'가 들어간 리뷰만 추림
opinion_sushi= [(review,voted_up,votes_up) for review,voted_up,votes_up in zip(filtered_data['review'],filtered_data['voted_up'],filtered_data['votes_up']) if '초밥' in review or '스시' in review]
opinion_sushi = pd.DataFrame.from_dict(opinion_sushi)
opinion_sushi

,0,1,2
0,모험 도트 아케이드를 좋아하는 제가 다이브를 경영 요리 파밍게임을 좋아하는 아내가 ...,True,211
1,초반인데 재미있네요 와이프랑도 할수 있게 멀티 지원 부탁드립니다 와이프는 식당에서 ...,True,288
2,다이빙이 맛있어보이고 스시가 즐거워보여요,True,88
3,재료공수 내가하고 서빙도 내가하고 와사비도 내가갈고 녹차도 내가따르고 맥주도 내가따...,True,62
4,아기자기한 힐링겜이지만 주인공에게 너무 가혹한 스토리인게 아쉬움 주인공이 너무 수동...,True,59
...,...,...,...
266,바라던것 하하호호 아름다운 바다탐험 하게된것 물고기 사냥과 스시 타이쿤 하지만 오랜...,True,1
267,얼엑 시간 후기 난 이 게임 내년 도 가능하다고 봄 넥슨인데 인디장르로 장르 불문하...,True,3
268,재밌당 근데 내 컴만 그런지 모르겠는데 맥주랑 차 따라줄때 프레임드랍이 살짝 있음 ...,True,0
269,상당히 잘 만든 게임입니다 개발자의 열정이 느껴지는 게임이었네요 그래픽도 독특하면서...,True,0


In [ ]:
# 총 추천 구하기
sum(opinion_sushi[2])

896

### 긍정적 리뷰

In [ ]:
# 긍정적 리뷰만 추림 (268개)
opinion_sushi_plus = opinion_sushi[opinion_sushi[1] == True]
opinion_sushi_plus

# 어구 추출 함수 사용
def remove_before_sushi(text):
  matches = re.finditer(r'초밥|스시',text)
  results_sushi = []
  for match in matches :
    start_index = match.start() - 5
    end_index = match.end() + 25
    if start_index < 0 :
      start_index = 0
    if end_index > len(text) :
      end_index = len(text)
    result = text[start_index:end_index]
    results_sushi.append(result)
  return results_sushi

# 결과를 새로운 열에 저장
opinion_sushi_plus['긍정'] = opinion_sushi_plus[0].apply(remove_before_sushi)
opinion_sushi_plus

<ipython-input-31-8c44f6f0b9d7>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_sushi_plus['긍정'] = opinion_sushi_plus[0].apply(remove_before_sushi)


,0,1,2,긍정
0,모험 도트 아케이드를 좋아하는 제가 다이브를 경영 요리 파밍게임을 좋아하는 아내가 ...,True,211,"[내가 반쇼스시 영업을 했습니다 함께 자리를 번갈아가면서 즐, 어종의 초밥과 요리..."
1,초반인데 재미있네요 와이프랑도 할수 있게 멀티 지원 부탁드립니다 와이프는 식당에서 ...,True,288,[식당에서 초밥팔고 저는 고기 잡으러 갈겁니다]
2,다이빙이 맛있어보이고 스시가 즐거워보여요,True,88,[어보이고 스시가 즐거워보여요]
3,재료공수 내가하고 서빙도 내가하고 와사비도 내가갈고 녹차도 내가따르고 맥주도 내가따...,True,62,[ 얻는건 초밥뿐이야]
4,아기자기한 힐링겜이지만 주인공에게 너무 가혹한 스토리인게 아쉬움 주인공이 너무 수동...,True,59,[데 입에 초밥한알 넣는 장면이 없음]
...,...,...,...,...
266,바라던것 하하호호 아름다운 바다탐험 하게된것 물고기 사냥과 스시 타이쿤 하지만 오랜...,True,1,"[ 사냥과 스시 타이쿤 하지만 오랜만의 타이쿤 오히려 좋아 , 데이브도 초밥집도 키..."
267,얼엑 시간 후기 난 이 게임 내년 도 가능하다고 봄 넥슨인데 인디장르로 장르 불문하...,True,3,"[다이브는 초밥경영 게임이라는 장르에 깊이를 부여함 초밥 만, 부여함 초밥 만드는..."
268,재밌당 근데 내 컴만 그런지 모르겠는데 맥주랑 차 따라줄때 프레임드랍이 살짝 있음 ...,True,0,[다 내가 초밥이 되게 생겼어]
269,상당히 잘 만든 게임입니다 개발자의 열정이 느껴지는 게임이었네요 그래픽도 독특하면서...,True,0,[ 다이빙 스시 반복이라는 게임 패턴을 어떻게 여러가지 바리]


In [ ]:
# 긍정적 리뷰 키워드 분석 진행
tokenizer = Okt()
opinion_sushi_plus['긍정'] = opinion_sushi_plus['긍정'].apply(lambda x: ', '.join(x))
opinion_sushi_plus['리뷰 분석 형태소'] = opinion_sushi_plus['긍정'].apply(tokenizer.morphs)
# 추출한 형태소 키워드 분석 진행
opinion_sushi_plus_mor = np.hstack(opinion_sushi_plus['리뷰 분석 형태소'].values)
# 단어의 빈도수를 계산
opinion_sushi_plus_mor = Counter(opinion_sushi_plus_mor)
opinion_sushi_plus_mor = Counter({x : opinion_sushi_plus_mor[x] for x in opinion_sushi_plus_mor if x not in stop_words})
opinion_sushi_plus_mor = sorted(opinion_sushi_plus_mor.items(), key=lambda x:x[1], reverse=True)
# 10번 이상 언급된 키워드만 출력
opinion_sushi_plus_mor = {word : count for word, count in opinion_sushi_plus_mor if count>= 10}
opinion_sushi_plus_mor

# 나도, 줘라는 단어가 비슷한 빈도로 언급 되는 것으로 보아 초밥을 만들어 판매하지만 주인공이 먹는 부분은 없어 유저들이 아쉬워하는 것으로 예상

<ipython-input-122-3dc321e7e2ef>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_sushi_plus['긍정'] = opinion_sushi_plus['긍정'].apply(lambda x: ', '.join(x))
<ipython-input-122-3dc321e7e2ef>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_sushi_plus['리뷰 분석 형태소'] = opinion_sushi_plus['긍정'].apply(tokenizer.morphs)


{'초밥': 343,
 '줘': 124,
 '집': 123,
 '나도': 122,
 '스시': 121,
 ',': 88,
 '운영': 58,
 '하는': 26,
 '에서': 21,
 '하고': 19,
 '재미': 18,
 '과': 17,
 '장사': 16,
 '만': 15,
 '에는': 15,
 '타이': 15,
 '밤': 14,
 '컨텐츠': 14,
 '물고기': 13,
 '사냥': 13,
 '쿤': 13,
 '경영': 12,
 '와': 11,
 '이나': 11,
 '할': 10,
 '다이빙': 10,
 '나': 10,
 '먹고': 10,
 '타이쿤': 10,
 '하': 10,
 '저녁': 10}

In [ ]:
# 추천 상위 10개 확인
opinion_sushi_plus.sort_values(by=2, ascending=False).head(10)

# 멀티 지원 요청 : 2 | 288+211
# 주인공이 초밥을 먹는 장면을 요청 : 2 | 59+14
# 초밥에 대한 긍정적 반응 : 5 | 88+62+15+13+9
# 기타 (분량에 대한 아쉬움) : 1 | 7

,0,1,2,긍정,리뷰 분석 형태소
1,초반인데 재미있네요 와이프랑도 할수 있게 멀티 지원 부탁드립니다 와이프는 식당에서 ...,True,288,식당에서 초밥팔고 저는 고기 잡으러 갈겁니다,"[식당, 에서, 초밥, 팔고, 저, 는, 고기, 잡으러, 갈, 겁니다]"
0,모험 도트 아케이드를 좋아하는 제가 다이브를 경영 요리 파밍게임을 좋아하는 아내가 ...,True,211,"내가 반쇼스시 영업을 했습니다 함께 자리를 번갈아가면서 즐, 어종의 초밥과 요리를...","[내, 가, 반, 쇼, 스시, 영업, 을, 했습니다, 함께, 자리, 를, 번갈아, ..."
2,다이빙이 맛있어보이고 스시가 즐거워보여요,True,88,어보이고 스시가 즐거워보여요,"[어보, 이고, 스시, 가, 즐거워, 보여요]"
3,재료공수 내가하고 서빙도 내가하고 와사비도 내가갈고 녹차도 내가따르고 맥주도 내가따...,True,62,얻는건 초밥뿐이야,"[얻는건, 초밥, 뿐, 이야]"
4,아기자기한 힐링겜이지만 주인공에게 너무 가혹한 스토리인게 아쉬움 주인공이 너무 수동...,True,59,데 입에 초밥한알 넣는 장면이 없음,"[데, 입, 에, 초밥, 한, 알, 넣는, 장면, 이, 없음]"
6,해양동물 사랑함 초밥 좋아함 프리 다이빙 가끔 함 그래서 게임영상만 보고 바로 내돈...,True,15,"사랑함 초밥 좋아함 프리 다이빙 가끔 함 그래서 게임영상, 기 보고 초밥도 만들고...","[사랑, 함, 초밥, 좋아함, 프리, 다이빙, 가끔, 함, 그래서, 게임, 영상, ..."
5,반쵸형님 초밥한입만 주이소 내가잡아온거 아입니까,True,14,반쵸형님 초밥한입만 주이소 내가잡아온거 아입니까,"[반, 쵸, 형님, 초밥, 한, 입, 만, 주이소, 내, 가, 잡아, 온거, 아, ..."
9,얼리인데 아직까지는 괜찮네요 정식버전이 어떨란지 모르겠지만 기대되는 게임입니다 반초...,True,13,니다 반초스시 조이고,"[니다, 반초, 스시, 조이, 고]"
8,풀어진 한국게임계에 긴장감을 불어넣는게임 오직 원초적인 게임성으로만 재미를 느끼게 ...,True,9,밤에는 스시집으로 게임 컨셉 자체가 신선함 얼리액세스 단,"[밤, 에는, 스시, 집, 으로, 게임, 컨셉, 자체, 가, 신선함, 얼리, 액세스..."
7,얼리인데도 정가로 사기에 아깝지 않아요 게임 할 때 천천히 전부 맛보는편이라 플탐 ...,True,7,끝냈고 초밥이랑 무기 다 만렙으로 만들어놓고 싶었으나 나,"[끝냈고, 초밥, 이랑, 무기, 다, 만렙, 으로, 만들어놓고, 싶었으나, 나]"


### 부정적 리뷰

In [ ]:
# 부정적 리뷰만 추림 (3개)
opinion_sushi_minus = opinion_sushi[opinion_sushi[1] != True]
# 결과를 새로운 열에 저장
opinion_sushi_minus['부정'] = opinion_sushi_minus[0].apply(remove_before_sushi)
opinion_sushi_minus
# 추천 상위 10개 확인
opinion_sushi_minus.sort_values(by=2,ascending=False).head(10)
# 컨텐츠의 지루함과 같은 부정적 의견 : 3 | 12

<ipython-input-38-bd438074ef6b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_sushi_minus['부정'] = opinion_sushi_minus[0].apply(remove_before_sushi)


,0,1,2,부정
52,한국 넥슨에서 만든 게임 친구들 사이에서도 궁금하다는 말이 많아 구매함 낮에는 바다 다이빙으로 물고기를 잡고 밤에는 초밥가게 타이쿤을 한다는 내용 한 번 켜봤다가 시간 내리 할 정도로 몰입력 있으나 아쉬운 부분도 많다 기껏 한국에서 만들어놓고 등장인물은 온통 일본인 일색 초밥이 아닌 스시라 표현한 것 부터 눈치챘지만 온갖 스테레오 타입 피규어에 집착하는 안여돼 오타쿠 권위있는 일남 따라다니는 젊은 여자 리포터 납치당하는 분홍돌고래와 남족장의 딸 등 과,False,10,"[ 밤에는 초밥가게 타이쿤을 한다는 내용 한 번 켜봤다가 시, 인 일색 초밥이 아닌 스시라 표현한 것 부터 눈치챘지만 온, 이 아닌 스시라 표현한 것 부터 눈치챘지만 온갖 스테레오 ]"
70,압도적 긍정적이라길래 해봤는데 년전에 폰으로 하던 게임 수준이다 그당시엔 할게 그런것밖에 없었고 게임 경험도 적었으니 그정도라도 만족했지만 할 게임이 많아지고 온갖 컨텐츠가 포함된 게임을 많이 겪다보니 이건 뭐 초딩들이나 좋아할법한 단순한 게임이라고 본다 것보다 물고기를 잡으려고 작살을 몇번 쏘는데 벌써부터 너무나 질리더라 너무나도 식상하고 그저 뻔한 패턴이라 나도 모르게 그만 또한 횟집에서 초밥을 나르고 녹차를 주는 것 역시 너무나 질리더라 이것 역시,False,2,[횟집에서 초밥을 나르고 녹차를 주는 것 역시 너무나 질리더]
75,경영이나 타이쿤류를 하고 싶다면 차라리 카이로소프트 게임 사서 하셈 나는 물질하고 경영 하는걸 하고싶어서 샀고 그 부분에 대해서는 충분히 재미있음 근데 그래도 어느정도 진행을 해야 뭐가 열릴테니까 스토리 미는데 짜증만 남 하프 스시새 끼들 말하는 싸가지도 불쾌하기만 하고 계속 심부름만 시켜먹고 그리고 겜을 재밌게 만들려고 미니게임도 이것저것 넣고 퍼즐도 이것저것 넣고 한거같은데 그게 너무 과함 특히 요리 대결 미니게임 하다가 진짜 열불나서 죽는줄 알았음,False,0,[남 하프 스시새 끼들 말하는 싸가지도 불쾌하기만 하고 계속]


## 5위 : 물고기 ( 232회 )

In [ ]:
# '물고기'가 들어간 리뷰만 추림 (187개)
opinion_fish= [(review,voted_up,votes_up) for review,voted_up,votes_up in zip(filtered_data['review'],filtered_data['voted_up'],filtered_data['votes_up']) if '물고기' in review ]
opinion_fish = pd.DataFrame.from_dict(opinion_fish)
opinion_fish

,0,1,2
0,물고기가 친절하고 사장님이 맛있어요,True,140
1,중간에 뭔 이상한 애니노래 같은거 나오고 뭔 노란 뚱뚱한 캐릭터가 야광봉 드는 미니...,True,50
2,이 게임은 그냥 향후 가게 분점으로 다른 해양과 다른 물고기만 늘어놔도 컨텐츠가 늘...,True,28
3,할인 기다리다가 지쳐서 그냥 원가주고 샀는데 원가 주고 살만한 게임이네요 스토리도 ...,True,14
4,해양동물 사랑함 초밥 좋아함 프리 다이빙 가끔 함 그래서 게임영상만 보고 바로 내돈...,True,15
...,...,...,...
182,도트도 예쁘고 게임도 너무 재미있고 좋습니다 물고기도 다양하고 가게운영도 재밌구요 ...,True,0
183,데모때부터 기대했었는데 상당히 잘나왔다 패러디도 여러곳에 있어서 패러디 보는맛도 있...,True,0
184,바라던것 하하호호 아름다운 바다탐험 하게된것 물고기 사냥과 스시 타이쿤 하지만 오랜...,True,1
185,이야 이거 재밌다 바라는 게 있다면 작살로 물고기 잡을 때 힘주는 키가 스페이스가 ...,True,0


In [ ]:
# 총 추천 구하기
sum(opinion_fish[2])

347

### 긍정적 리뷰

In [ ]:
# 긍정적 리뷰만 추림 (178개)
opinion_fish_plus = opinion_fish[opinion_fish[1] == True]
opinion_fish_plus
def remove_before_fish(text):
    matches = re.finditer(r'물고기', text)
    results_fish = []
    for match in matches :
      start_index = match.start() - 5
      end_index = match.end() + 25
      if start_index < 0 :
        start_index = 0
      if end_index > len(text) :
        end_index = len(text)
      result = text[start_index:end_index]
      results_fish.append(result)
    return results_fish
# 결과를 새로운 열에 저장
opinion_fish_plus['긍정'] = opinion_fish_plus[0].apply(remove_before_fish)
# 긍정 리뷰 키워드 분석 진행
tokenizer = Okt()
opinion_fish_plus['긍정'] = opinion_fish_plus['긍정'].apply(lambda x:', '.join(x))
opinion_fish_plus['리뷰 분석 형태소'] = opinion_fish_plus['긍정'].apply(tokenizer.morphs)
# 추출한 형태소 키워드 분석 진행
opinion_fish_plus_mor = np.hstack(opinion_fish_plus['리뷰 분석 형태소'].values)
# 단어의 빈도수를 계산
opinion_fish_plus_mor = Counter(opinion_fish_plus_mor)
opinion_fish_plus_mor = Counter({x : opinion_fish_plus_mor[x] for x in opinion_fish_plus_mor if x not in stop_words})
opinion_fish_plus_mor = sorted(opinion_fish_plus_mor.items(), key=lambda x:x[1], reverse=True)
# 10번 이상 언급된 키워드만 출력
opinion_fish_plus_mor = {word : count for word, count in opinion_fish_plus_mor if count>= 10}
opinion_fish_plus_mor
# 키워드 분석 결과 특정 의견을 예상할 수 없음

<ipython-input-44-2e2d4d6762d4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_fish_plus['긍정'] = opinion_fish_plus[0].apply(remove_before_fish)
<ipython-input-44-2e2d4d6762d4>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_fish_plus['긍정'] = opinion_fish_plus['긍정'].apply(lambda x:', '.join(x))
<ipython-input-44-2e2d4d6762d4>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

{'물고기': 234,
 ',': 43,
 '잡고': 34,
 '초밥': 30,
 '집': 28,
 '운영': 20,
 '하는': 20,
 '스시': 20,
 '만': 19,
 '하고': 17,
 '잡는': 17,
 '재미': 14,
 '과': 12,
 '나': 12,
 '사냥': 12,
 '히': 12,
 '에서': 11,
 '잡은': 10,
 '돈': 10,
 '종류': 10,
 '에는': 10}

In [ ]:
# 추천 상위 10개 확인
opinion_fish_plus.sort_values(by=2,ascending=False).head(10)

# 물고기 사냥에 대한 긍정적 의견 : 1 | 14
# 기타 (키워드와 관련없는 내용) : 5 | 50+28+7+4+3
# 물고기에 대한 긍정적 반응 : 2 | 140+15
# 버그 및 작동 방식에 대한 리뷰 : 2 | 4+3

,0,1,2,긍정,리뷰 분석 형태소
0,물고기가 친절하고 사장님이 맛있어요,True,140,물고기가 친절하고 사장님이 맛있어요,"[물고기, 가, 친절하고, 사장, 님, 이, 맛있어요]"
1,중간에 뭔 이상한 애니노래 같은거 나오고 뭔 노란 뚱뚱한 캐릭터가 야광봉 드는 미니...,True,50,도 그냥 물고기 잡고 평범하게 가게차리는 그런건줄 알았지,"[도, 그냥, 물고기, 잡고, 평범하게, 가게, 차리는, 그런건, 줄, 알았지]"
2,이 게임은 그냥 향후 가게 분점으로 다른 해양과 다른 물고기만 늘어놔도 컨텐츠가 늘...,True,28,과 다른 물고기만 늘어놔도 컨텐츠가 늘것이고 스토리상으로는,"[과, 다른, 물고기, 만, 늘어놔도, 컨텐츠, 가, 늘것이고, 스토리, 상, 으로는]"
4,해양동물 사랑함 초밥 좋아함 프리 다이빙 가끔 함 그래서 게임영상만 보고 바로 내돈...,True,15,는 나는 물고기 보고 초밥도 만들고 아기자기하니 좋아요 근데,"[는, 나, 는, 물고기, 보고, 초밥, 도, 만들고, 아기자기하니, 좋아요, 근데]"
3,할인 기다리다가 지쳐서 그냥 원가주고 샀는데 원가 주고 살만한 게임이네요 스토리도 ...,True,14,고 일단 물고기 잡는게 재밌어서 시간 가는 줄 모르겠어요,"[고, 일단, 물고기, 잡는게, 재밌어서, 시간, 가는, 줄, 모르겠어요]"
5,정식 출시가 기다려지는 잔잔하고 유머러스한 파밍 피슁 게임 바닷속을 아기자기하면서도...,True,7,"도 잡은 물고기들을 초밥으로 판매하기 위해 요리를 연구하고 , 을 통해 물고기를 키...","[도, 잡은, 물고기, 들, 을, 초밥, 으로, 판매, 하기, 위해, 요리, 를, ..."
8,벽에 끼인 상어 파밍할 때 영원히 파밍하면서 취소도 이동도 못하는 버그가 있습니다 ...,True,4,파밍한 물고기들 다 잃어서 많이 슬프네요 빨리 고쳐졌으면,"[파밍, 한, 물고기, 들, 다, 잃어서, 많이, 슬프네요, 빨리, 고쳐졌으면]"
28,바다속에서 물고기도 잡고 잡템도 줍고 잡은 물고기로 초밥집을 운영하는 게임 타이쿤 ...,True,4,"다속에서 물고기도 잡고 잡템도 줍고 잡은 물고기로 초밥집을 , 고 잡은 물고기로 초...","[다, 속, 에서, 물고기, 도, 잡고, 잡템, 도, 줍고, 잡은, 물고기, 로, ..."
6,물고기 사냥 채집과 업그레이드 카드 수집 식당 운영 직원 육성 양식장 관리 퀘스트 ...,True,3,물고기 사냥 채집과 업그레이드 카드 수집 식당 운영,"[물고기, 사냥, 채집, 과, 업그레이드, 카드, 수집, 식당, 운영]"
7,시간 정도 했고 챕터 끝냈음 지금까지는 굉장히 마음에 듬 캐릭터가 이미 비쥬얼로 특...,True,3,작살로 물고기 잡는거 첨에 손가락 아파서,"[작살, 로, 물고기, 잡는거, 첨, 에, 손가락, 아파서]"


### 부정적 리뷰

In [ ]:
# 부정적 리뷰만 추림 (9개)
opinion_fish_minus = opinion_fish[opinion_fish[1] != True]
# 결과를 새로운 열에 저장
opinion_fish_minus['부정'] = opinion_fish_minus[0].apply(remove_before_fish)
opinion_fish_minus
# 추천 상위 10개 확인
opinion_fish_minus.sort_values(by=2,ascending=False).head(10)
# 기타 (키워드와 관련 없는 내용) : 2 | 10+3
# 물고기가 컨텐츠의 전부인 것 같아 아쉬워 하는 리뷰 : 1 | 5
# 물고기 사냥 시스템에 대한 불만 : 2 | 2+1
# 버그에 대한 불만 : 1 | 0
# 물고기 종류 부족에 대한 불만 : 1 | 0
# 무게 한도에 대한 불만 : 1 | 1

<ipython-input-142-4760ae9c5a34>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_fish_minus['부정'] = opinion_fish_minus[0].apply(remove_before_fish)


,0,1,2,부정
32,한국 넥슨에서 만든 게임 친구들 사이에서도 궁금하다는 말이 많아 구매함 낮에는 바다...,False,10,[이빙으로 물고기를 잡고 밤에는 초밥가게 타이쿤을 한다는 내용]
27,각 잡고 하면 다섯 시간도 안 나올 플레임타임에 원이라는 가격은 아직 비싸다 원짜리...,False,5,[기자기한 물고기들을 보는 맛이 전부인 느낌이다 서브노티카를 ]
36,뭔가 프로 부족한 게임 메인 컨텐츠인 물고기 잡기 회떠서 팔기 자체는 재밌는데 후반...,False,3,[컨텐츠인 물고기 잡기 회떠서 팔기 자체는 재밌는데 후반부도 ]
48,압도적 긍정적이라길래 해봤는데 년전에 폰으로 하던 게임 수준이다 그당시엔 할게 그런...,False,2,[ 것보다 물고기를 잡으려고 작살을 몇번 쏘는데 벌써부터 너무]
43,아직 정식오픈이 아니라 그런지 뭔가 부족한 느낌과 답답하고 반복되는 지겨움 게임내 ...,False,1,[도 없고 물고기 몇마리 잡고나면 무게 때문에 복귀해야하고 사]
49,내가씨발 잠수하면서 물고기잡고 횟집운영하는 게임하려고 샀지 좆같은 선공물고기때문에 ...,False,1,"[수하면서 물고기잡고 횟집운영하는 게임하려고 샀지 좆같은 선공, 같은 선공물고기때문..."
12,제작진 하고싶은거 다 넣은것같은 게임 미니게임도 많고 컨텐츠도 많지만 아쉬움도 많은...,False,0,[임입니다 물고기 사냥시간에 비해 장사하는 시간이 너무 짧게 ]
44,아니 시발 왜 게임중에 자꾸 튕기냐고 니들 업데이트 하는 거랑 별개로 게임은 계속 ...,False,0,[ 열심히 물고기 잡고 이것 저것 줏었는데 튕겼네 씨발 이게 ]
131,물고기 종류도 너무 없고 밤에 장사해도 너무 빨리끝나버리고 돈도 많이 못 벌고 빨리...,False,0,[물고기 종류도 너무 없고 밤에 장사해도 너무 빨리끝]


## 공동 6위 : 재미 ( 217회 )

In [ ]:
# '재미'가 들어간 리뷰만 추림 (435개)
opinion_fun= [(review,voted_up,votes_up) for review,voted_up,votes_up in zip(filtered_data['review'],filtered_data['voted_up'],filtered_data['votes_up']) if '재미' in review ]
opinion_fun = pd.DataFrame.from_dict(opinion_fun)
opinion_fun

,0,1,2
0,초반인데 재미있네요 와이프랑도 할수 있게 멀티 지원 부탁드립니다 와이프는 식당에서 ...,True,288
1,넥슨게임에서 과금제도를 빼버리니 더 재미있다,True,83
2,자극적이지 않고 재미있는 게임 정말 오랜만이야,True,69
3,너무 재미있어요 컨텐츠 거의다해서 정출만 기다리는 중인데 혹시 정식출시되면 원래 세...,True,29
4,평소 넥슨게임 하면 안좋은 이미지로 가득차 있었는데 믿기지 않을정도로 재미있고 잘만...,True,42
...,...,...,...
430,피처폰으로 타이쿤게임하던 그감성 오랜만에 느꼈음 굉장히 재미있음 근데 얼액감안하더라...,True,0
431,미니게임 퍼즐 이거저거 굉장히 많은데 진짜 재미가 하나도없음 젤다 튜닉 엄청 재밌게...,False,0
432,재미있네요,True,2
433,재밌다 출근 해야되기 떄문에 자러가야된다는 사실이 쩝 얼리액세스 치고 그래도 재미있...,True,0


In [ ]:
# 총 추천
sum(opinion_fun[2])

747

### 긍정적 리뷰

In [ ]:
# 긍정적 리뷰만 추림 (420개)
opinion_fun_plus = opinion_fun[opinion_fun[1] == True]
opinion_fun_plus
def remove_before_fun(text):
    matches = re.finditer(r'재미', text)
    results_fun = []
    for match in matches :
      start_index = match.start() - 5
      end_index = match.end() + 25
      if start_index < 0 :
        start_index = 0
      if end_index > len(text) :
        end_index = len(text)
      result = text[start_index:end_index]
      results_fun.append(result)
    return results_fun
# 결과를 새로운 열에 저장
opinion_fun_plus['긍정'] = opinion_fun_plus[0].apply(remove_before_fun)
# 긍정 리뷰 키워드 분석 진행
tokenizer = Okt()
opinion_fun_plus['긍정'] = opinion_fun_plus['긍정'].apply(lambda x:', '.join(x))
opinion_fun_plus['리뷰 분석 형태소'] = opinion_fun_plus['긍정'].apply(tokenizer.morphs)
# 추출한 형태소 키워드 분석 진행
opinion_fun_plus_mor = np.hstack(opinion_fun_plus['리뷰 분석 형태소'].values)
# 단어의 빈도수를 계산
opinion_fun_plus_mor = Counter(opinion_fun_plus_mor)
opinion_fun_plus_mor = Counter({x : opinion_fun_plus_mor[x] for x in opinion_fun_plus_mor if x not in stop_words})
opinion_fun_plus_mor = sorted(opinion_fun_plus_mor.items(), key=lambda x:x[1], reverse=True)
# 10번 이상 언급된 키워드만 출력
opinion_fun_plus_mor = {word : count for word, count in opinion_fun_plus_mor if count>= 10}
opinion_fun_plus_mor
# 키워드 분석 결과 '재미있는', '재미있게','재미있어요', '재미있음','재미있고', '재미있습니다'와 같은 재미있다는 내용이 대부분
# 분석 결과로 추정하면 컨텐츠, 스토리, 탐험, 경영, 타이쿤, 운영, 물고기와 같은 세부 요소를 재밌게 즐긴 것으로 예상

<ipython-input-46-283537560621>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_fun_plus['긍정'] = opinion_fun_plus[0].apply(remove_before_fun)
<ipython-input-46-283537560621>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_fun_plus['긍정'] = opinion_fun_plus['긍정'].apply(lambda x:', '.join(x))
<ipython-input-46-283537560621>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

{'재미': 224,
 ',': 97,
 '하는': 47,
 '재미있는': 43,
 '정식': 41,
 '재미있게': 37,
 '재미있어요': 36,
 '재미있음': 36,
 '재미있습니다': 31,
 '재미있고': 26,
 '출시': 26,
 '재미있다': 25,
 '컨텐츠': 21,
 '과': 19,
 '하고': 19,
 '입니다': 18,
 '재미있네요': 17,
 '만': 17,
 '와': 17,
 '있는': 17,
 '아직': 16,
 '할': 15,
 '스토리': 15,
 '적': 15,
 '까지': 14,
 '탐험': 14,
 '물고기': 13,
 '운영': 13,
 '얼리': 12,
 '기대': 11,
 '에서': 11,
 '되는': 11,
 '잘': 11,
 '초밥': 11,
 '있고': 11,
 '진': 11,
 '서': 10,
 '재미있는데': 10,
 '요': 10,
 '요소': 10,
 '타이쿤': 10,
 '매우': 10,
 '어인': 10}

In [ ]:
# 추천 상위 10개 확인
opinion_fun_plus.sort_values(by=2,ascending=False).head(10)

# 10개 모두 게임에 대한 긍정적인 반응

,0,1,2,긍정,리뷰 분석 형태소
0,초반인데 재미있네요 와이프랑도 할수 있게 멀티 지원 부탁드립니다 와이프는 식당에서 초밥팔고 저는 고기 잡으러 갈겁니다,True,288,초반인데 재미있네요 와이프랑도 할수 있게 멀티 지원 부탁드,"[초반, 인데, 재미있네요, 와이, 프랑, 도, 할수, 있게, 멀티, 지원, 부탁드]"
1,넥슨게임에서 과금제도를 빼버리니 더 재미있다,True,83,리니 더 재미있다,"[리니, 더, 재미있다]"
2,자극적이지 않고 재미있는 게임 정말 오랜만이야,True,69,지 않고 재미있는 게임 정말 오랜만이야,"[지, 않고, 재미있는, 게임, 정말, 오랜, 만이, 야]"
4,평소 넥슨게임 하면 안좋은 이미지로 가득차 있었는데 믿기지 않을정도로 재미있고 잘만든 수작인것같다 만원 가격치고 컨탠츠가 풍부하고 도트애니메이션에 정성을 가득 쏟아부은게 느껴진다 아직 정식 출시가 아님에도 이정도 재미를 느낄수있는건 기대해볼만하다고 생각한다 즐거움을 느낄수있는 게임을 만들 줄 알면서 왜 여태 이래왔던거냐 넥슨,True,42,"을정도로 재미있고 잘만든 수작인것같다 만원 가격치고 컨탠츠, 이정도 재미를 느낄수있는건 기대해볼만하다고 생각한다 즐거","[을, 정도, 로, 재미있고, 잘만, 든, 수작, 인, 것, 같다, 만원, 가격, 치고, 컨탠츠, ,, 이정, 도, 재미, 를, 느낄수있는건, 기대, 해볼만, 하다고, 생각, 한다, 즐거]"
3,너무 재미있어요 컨텐츠 거의다해서 정출만 기다리는 중인데 혹시 정식출시되면 원래 세이브파일 유지되나요,True,29,너무 재미있어요 컨텐츠 거의다해서 정출만 기다리는 중인,"[너무, 재미있어요, 컨텐츠, 거의, 다해, 서, 정, 출만, 기다리는, 중인]"
5,데모 플레이 후 기다리고 있던 게임입니다 좋아하는 장르여서 기대를 한 것도 있지만 넥슨 에서 만들었다는 것에 기대를 더 하고 있었습니다 민트 로켓은 수익성이 아니라 순수 재미를 위한 게임을 개발하겠다고 설립한 넥슨의 서브 브랜드입니다 대기업에서도 이런 장르의 게임들을 만들어 나간다면 앞으로 국산 게임이 좀 더 다양해질 수도 있지 않을까 하는 기대감 때문이었죠 그리고 출시하자마자 버그 관련 리뷰가 있어서 걱정했는데 다행히도 저는 하루 플레이하는 동안 버그,True,16,라 순수 재미를 위한 게임을 개발하겠다고 설립한 넥슨의 서,"[라, 순수, 재미, 를, 위, 한, 게임, 을, 개발, 하겠다고, 설립, 한, 넥슨, 의, 서]"
7,진짜 너무 재미있는데 정식런칭 앞의 퀘스트까지 시간이면 충분하고 지금 갈 수 있는 수준에서 나오는 상어도 다 금방 잡음 정식 런칭 언제,True,15,짜 너무 재미있는데 정식런칭 앞의 퀘스트까지 시간이면 충분,"[짜, 너무, 재미있는데, 정식, 런칭, 앞, 의, 퀘스트, 까지, 시간, 이면, 충분]"
11,일단 갓 겜 이라고 외치고 시작하겠습니다 재미는 확실히 보장합니다 장점부터 말하겠습니다 굉장히 편안한 노래 도트감성 최고 아침엔 파밍 저녁에는 장사 타이쿤과 파밍의 재미를 동시에 느낄수있음 캐릭터들 하나하나 매력이 넘쳐나요 다양한 총기 아직 얼리엑세스라 조금 부족한감도 있긴하지만 지금도 충분히 활용성이 무궁무진합니다 쿡스타그램 너무 귀여워요 반쵸아저씨 너무사랑해 단점 버그가 조오금 있습니다 단점은 이거하나 다른곳에서 단점을 느낄수없었습니다 그리고 마지막,True,12,"겠습니다 재미는 확실히 보장합니다 장점부터 말하겠습니다 굉, 파밍의 재미를 동시에 느낄수있음 캐릭터들 하나하나 매력이","[겠습니다, 재미, 는, 확실히, 보장, 합니다, 장점, 부터, 말, 하겠습니다, 굉, ,, 파밍, 의, 재미, 를, 동시, 에, 느낄수있음, 캐릭터, 들, 하나, 하나, 매력, 이]"
6,재미있고 컨텐츠도 많아보이나 실제해보면 플레이타임이 상당히 짧은편,True,11,재미있고 컨텐츠도 많아보이나 실제해보면 플레이타임,"[재미있고, 컨텐츠, 도, 많아, 보이, 나, 실제, 해보면, 플레이, 타임]"
15,과대평가 이것 저것 할 게 있지만 다 재미있는 것은 아님 얼리액세스라는 걸 감안해서 넘어가지만 볼륨이 많이 부족함 응원하는 김에 추천으로 해 놨지만 이 상태로는 추천 못 함,True,10,지만 다 재미있는 것은 아님 얼리액세스라는 걸 감안해서 넘,"[지만, 다, 재미있는, 것, 은, 아님, 얼리, 액세스, 라는, 걸, 감안, 해서, 넘]"


### 부정적 리뷰

In [ ]:
# 부정적 리뷰만 추림 (15개)
opinion_fun_minus = opinion_fun[opinion_fun[1] != True]
# 결과를 새로운 열에 저장
opinion_fun_minus['부정'] = opinion_fun_minus[0].apply(remove_before_fun)
# 부정적 리뷰 키워드 분석 진행
tokenizer = Okt()
opinion_fun_minus['부정'] = opinion_fun_minus['부정'].apply(lambda x: ', '.join(x))
opinion_fun_minus['리뷰 분석 형태소'] = opinion_fun_minus['부정'].apply(tokenizer.morphs)
# 추출한 형태소 키워드 분석 진행
opinion_fun_minus_mor = np.hstack(opinion_fun_minus['리뷰 분석 형태소'].values)
# 단어의 빈도수를 계산
opinion_fun_minus_mor = Counter(opinion_fun_minus_mor)
opinion_fun_minus_mor = Counter({x : opinion_fun_minus_mor[x] for x in opinion_fun_minus_mor if x not in stop_words})
opinion_fun_minus_mor = sorted(opinion_fun_minus_mor.items(), key=lambda x:x[1], reverse=True)
# 10번 이상 언급된 키워드만 출력
opinion_fun_minus_mor = {word : count for word, count in opinion_fun_minus_mor if count>= 2}
opinion_fun_minus_mor
# 키워드 분석 결과 재미없다고 생각하는 특정 요소를 추측하기는 어려움

<ipython-input-158-8ea062248279>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_fun_minus['부정'] = opinion_fun_minus[0].apply(remove_before_fun)
<ipython-input-158-8ea062248279>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_fun_minus['부정'] = opinion_fun_minus['부정'].apply(lambda x: ', '.join(x))
<ipython-input-158-8ea062248279>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

{'재미': 14,
 '있지만': 3,
 ',': 3,
 '있고': 2,
 '재미없음': 2,
 '고': 2,
 '적': 2,
 '데': 2,
 '해야': 2,
 '없음': 2,
 '글쎄': 2,
 '어릴': 2,
 '부터': 2,
 '근데': 2}

In [ ]:
# 추천 상위 10개 확인
opinion_fun_minus.sort_values(by=2,ascending=False).head(10)

# 기타 (키워드와 관련 없는 내용, 다른 부분이 부정적이므로 부정적 리뷰로 분류된 경우) : 4 | 7+2+2+1
# 재미없다고 밝힌 내용 : 6 | 4+4+3+2+1+1

,0,1,2,부정,리뷰 분석 형태소
19,처음 구매하고 시간정도는 잠수장비 업글하고 하면서 재미있게 했는데 얼리라서 소지금 ...,False,7,하면서 재미있게 했는데 얼리라서 소지금 상한도 있고 스토,"[하면서, 재미있게, 했는데, 얼리라서, 소, 지금, 상한, 도, 있고, 스토]"
21,별로 재미없음,False,4,별로 재미없음,"[별로, 재미없음]"
64,왜 나만 별루지 첨엔 우와 했는데 좀지나면 겁나 지루해 사람마다 다르겠지만 나는 재...,False,4,만 나는 재미 읍다 가격이 만원대 라면 압도적 긍정적 이해,"[만, 나, 는, 재미, 읍, 다, 가격, 이, 만원, 대, 라면, 압도, 적, 긍..."
62,뭔가 프로 부족한 게임 메인 컨텐츠인 물고기 잡기 회떠서 팔기 자체는 재밌는데 후반...,False,3,즈해지고 재미없어짐 퀘스트 오브젝트 찾는 것들 중이 그 원,"[즈, 해지, 고, 재미, 없어짐, 퀘스트, 오브젝트, 찾는, 것, 들, 중이, 그..."
77,얼리엑세스 기준 시간 클리어 가능한 볼륨 업데이트 생각이 없음 재미는 있지만 왜 벌...,False,2,이 없음 재미는 있지만 왜 벌써 오픈한건지 잘 모르겠음 게,"[이, 없음, 재미, 는, 있지만, 왜, 벌써, 오픈, 한, 건지, 잘, 모르겠음, 게]"
83,재미없음,False,2,재미없음,[재미없음]
61,재미를 떠나서 게임이 많이 불안정한듯 프로그램도 사양이나 그래픽에 비해 무거운 느낌...,False,1,재미를 떠나서 게임이 많이 불안정한듯 프로그램도,"[재미, 를, 떠나서, 게임, 이, 많이, 불안정한듯, 프로그램, 도]"
71,비주얼은 귀엽고 예쁜데 그닥 재미가 없었고 사냥시 마우스를 길게 홀딩해야해서 플레이...,False,1,데 그닥 재미가 없었고 사냥시 마우스를 길게 홀딩해야해서,"[데, 그닥, 재미, 가, 없었고, 사냥, 시, 마우스, 를, 길게, 홀딩, 해야,..."
78,재미없습니다,False,1,재미없습니다,[재미없습니다]
81,재미는있으나 역시 넥슨게임 넥슨타이틀을 벗어나지 못한 게임 같다 양산형게임들 보단 ...,False,1,"재미는있으나 역시 넥슨게임 넥슨타이틀을 벗어나지 , 들 보단 재미도 있고 컨셉도 좋...","[재미, 는, 있으나, 역시, 넥슨, 게임, 넥슨, 타이틀, 을, 벗어나지, ,, ..."


## 공동 6위 : 스토리 ( 217회 )

In [ ]:
# '스토리'가 들어간 리뷰만 추림(185개)
opinion_story= [(review,voted_up,votes_up) for review,voted_up,votes_up in zip(filtered_data['review'],filtered_data['voted_up'],filtered_data['votes_up']) if '스토리' in review ]
opinion_story = pd.DataFrame.from_dict(opinion_story)
opinion_story

,0,1,2
0,가격대비 정말 혜자게임인거는 맞지만 중간에 어인족 마을에서 노가다는 정말 짜증났습니...,True,99
1,치킨을 한 번 참을만한 가치가 있는 게임 평가 상단은 구매를 고려하는 분들을 위해 ...,True,102
2,다양한 시도를 한 넥슨 드디어 일냈다 꽤 뛰어난 도트퀄리티 자연스러운 연출 적당한 ...,True,99
3,아기자기한 힐링겜이지만 주인공에게 너무 가혹한 스토리인게 아쉬움 주인공이 너무 수동...,True,59
4,꼭 패드로 하세요 진동 지립니다 게임 이미지도 이쁘고 재밌고 스토리도 괜찮고 다 좋...,True,89
...,...,...,...
180,파밍 스토리 도트가 주는 감성까지 너무 재미있는 게임이네요 강추 다음업뎃이 기대됩니다,True,0
181,재미있습니다 심해에서 사냥하기와 식당운영하기 그외 여러가지 할게 많습니다 시간만에 ...,True,0
182,이쁜 그래픽 개성넘치는 캐릭터 재밌는 스토리와 연출 그리고 그 속에 담긴 사회풍자까...,True,0
183,작살 쏘는 손맛이 대박입니다 무기강화 잠수복강화 스시집 운영 등 다양한 성장요소가 ...,True,0


In [ ]:
# 총 추천 확
sum(opinion_story[2])

588

### 긍정적 리뷰

In [ ]:
# 긍정적 리뷰만 추림 (173개)
opinion_story_plus = opinion_story[opinion_story[1] == True]
def remove_before_story(text):
    matches = re.finditer(r'스토리', text)
    results_story = []
    for match in matches :
      start_index = match.start() - 10
      end_index = match.end() + 25
      if start_index < 0 :
        start_index = 0
      if end_index > len(text) :
        end_index = len(text)
      result = text[start_index:end_index]
      results_story.append(result)
    return results_story
# 결과를 새로운 열에 저장
opinion_story_plus['긍정'] = opinion_story_plus[0].apply(remove_before_story)
# 긍정 리뷰 키워드 분석 진행
tokenizer = Okt()
opinion_story_plus['긍정'] = opinion_story_plus['긍정'].apply(lambda x:', '.join(x))
opinion_story_plus['리뷰 분석 형태소'] = opinion_story_plus['긍정'].apply(tokenizer.morphs)
# 추출한 형태소 키워드 분석 진행
opinion_story_plus_mor = np.hstack(opinion_story_plus['리뷰 분석 형태소'].values)
# 단어의 빈도수를 계산
opinion_story_plus_mor = Counter(opinion_story_plus_mor)
opinion_story_plus_mor = Counter({x : opinion_story_plus_mor[x] for x in opinion_story_plus_mor if x not in stop_words})
opinion_story_plus_mor = sorted(opinion_story_plus_mor.items(), key=lambda x:x[1], reverse=True)
# 10번 이상 언급된 키워드만 출력
opinion_story_plus_mor = {word : count for word, count in opinion_story_plus_mor if count>= 10}
opinion_story_plus_mor
# 키워드 분석 결과 정식 출시를 기다리는 내용 혹은 진행한 내용까지의 리뷰가 대부분이라고 예상

<ipython-input-49-f6bb7ddbd218>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_story_plus['긍정'] = opinion_story_plus[0].apply(remove_before_story)
<ipython-input-49-f6bb7ddbd218>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_story_plus['긍정'] = opinion_story_plus['긍정'].apply(lambda x:', '.join(x))
<ipython-input-49-f6bb7ddbd218>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

{'스토리': 191,
 '진행': 21,
 ',': 20,
 '까지': 18,
 '정식': 17,
 '고': 15,
 '적': 15,
 '컨텐츠': 14,
 '출시': 13,
 '메인': 13,
 '만': 12,
 '나': 11,
 '초밥': 11,
 '하고': 11,
 '집': 11,
 '아직': 10,
 '와': 10,
 '과': 10}

In [ ]:
# 추천 상위 10개 확인
opinion_story_plus.sort_values(by=2,ascending = False).head(10)

# 스토리 부족 : 1 | 102
# 스토리 내 반복성에 대한 불만 : 2 | 99+15
# 스토리에 대한 만족 : 3 | 99+89+14
# 스토리에 대한 아쉬움 : 1 | 59
# 스토리 추가에 대한 기대 : 1 | 28
# 기타 : 2 | 9+7

,0,1,2,긍정,리뷰 분석 형태소
1,치킨을 한 번 참을만한 가치가 있는 게임 평가 상단은 구매를 고려하는 분들을 위해 플레이하며 느낀 장단점을 간략하게 적어놓고 평가 하단은 게임의 장단점을 조금 더 주관적 견해와 생각을 적었습니다 좋은 점 저렴한 가격 도트로 표현한 아름다운 해양 생태계 다양한 볼륨 및 컨텐츠 어렵지 않은 플레이 방식 매력 넘치는 캐릭터들 차분하고 어울리는 음악 아쉬운 점 얼리 엑세스 완성되지 않은 스토리 피할 수 없는 반복성 플레이 조금 어색한 조작감 키보드 마우스 자잘,True,102,스 완성되지 않은 스토리 피할 수 없는 반복성 플레이 조금 어색한 조,"[스, 완성, 되지, 않은, 스토리, 피, 할, 수, 없는, 반복, 성, 플레이, 조금, 어색한, 조]"
0,가격대비 정말 혜자게임인거는 맞지만 중간에 어인족 마을에서 노가다는 정말 짜증났습니다 마치 메이플스토리 퀘스트 노가다 마냥 계속 왔다리갔다리 시키는거 개극혐 근데 게임은 전체적으로 재밌습니다,True,99,습니다 마치 메이플스토리 퀘스트 노가다 마냥 계속 왔다리갔다리 시키는,"[습니다, 마치, 메이플스토리, 퀘스트, 노가다, 마냥, 계속, 왔다리, 갔다리, 시키는]"
2,다양한 시도를 한 넥슨 드디어 일냈다 꽤 뛰어난 도트퀄리티 자연스러운 연출 적당한 밸런스를 자랑하는 게임이면서 주인공 데이브를 통해 바다를 탐험하는 게 꽤 재밌었다 피는 좀 튀기지만 고어연출은 전혀 없으며 스토리 진행에 대한 밸런스도 꽤 맞는 편이고 얼리엑세스임에도 불구하고 꽤 즐길 컨텐츠가 꽤 많다 현재 양식장을 사용할 수 있는 게 임인 것을 보면 아직 게임은 밖에 나오지 않았다고 생각하면 후에 정식출시할 때 더 많은 컨텐츠가 등장할 것으로 보인다 낮,True,99,출은 전혀 없으며 스토리 진행에 대한 밸런스도 꽤 맞는 편이고 얼리엑,"[출은, 전혀, 없으며, 스토리, 진행, 에, 대한, 밸런스, 도, 꽤, 맞는, 편이, 고, 얼리, 엑]"
4,꼭 패드로 하세요 진동 지립니다 게임 이미지도 이쁘고 재밌고 스토리도 괜찮고 다 좋아요 근데 하나 아쉬운점이 있다면 분명 국내 개발게임인데 한글화가 제대로 안 된 느낌 번역이 이상하다 이런게 아니라 쓸데없는 곳에 한글대신 영어를 넣어놨어요 그것도 꽤 많이 예를 들면 깊이를 로 표현한다든지 굳이 외래어도 아니고 영어표기가 필요없는걸 넣어놓으니까 살짝 짜증나는 부분이 있습니다 마치 매표소를 라고 표시하는 느낌 물론 어려운 영어 아니고 조금만 익숙해지면 게임,True,89,도 이쁘고 재밌고 스토리도 괜찮고 다 좋아요 근데 하나 아쉬운점이 있,"[도, 이쁘고, 재밌고, 스토리, 도, 괜찮고, 다, 좋아요, 근데, 하나, 아쉬운, 점, 이, 있]"
3,아기자기한 힐링겜이지만 주인공에게 너무 가혹한 스토리인게 아쉬움 주인공이 너무 수동적이고 자는시간빼고 모든시간이 노동인데 입에 초밥한알 넣는 장면이 없음,True,59,에게 너무 가혹한 스토리인게 아쉬움 주인공이 너무 수동적이고 자는시간,"[에게, 너무, 가혹한, 스토리, 인게, 아쉬움, 주인공, 이, 너무, 수동, 적, 이고, 자는, 시간]"
5,이 게임은 그냥 향후 가게 분점으로 다른 해양과 다른 물고기만 늘어놔도 컨텐츠가 늘것이고 스토리상으로는 갑자기 마야문명 발견했다 해도 스토리 분량이 늘것이다 아니 그냥 아직 발닿지 않은 심해에 어떠한 설정을 넣어도 게임 컨텐츠는 마르지 않을 것이다,True,28,"컨텐츠가 늘것이고 스토리상으로는 갑자기 마야문명 발견했다 해도 스토리, 명 발견했다 해도 스토리 분량이 늘것이다 아니 그냥 아직 발닿지 않은","[컨텐츠, 가, 늘것이고, 스토리, 상, 으로는, 갑자기, 마야문명, 발견, 했다, 해도, 스토리, ,, 명, 발견, 했다, 해도, 스토리, 분량, 이, 늘것이다, 아니, 그냥, 아직, 발, 닿지, 않은]"
7,해양동물 사랑함 초밥 좋아함 프리 다이빙 가끔 함 그래서 게임영상만 보고 바로 내돈내산 했는데 제작사 보고 좀 놀랬어요 과거에 다양한 방식으로 내 돈 다 털어가던 곳이 이런 겜을 만들다니 그리고 지금도 내 돈을 털어가는 구나 낮에는 물질하고 밤에는 알바하고 누가봐도 노동부의 도움이 필요한 주인공에겐 가혹한 스케쥴이지만 게임하는 나는 물고기 보고 초밥도 만들고 아기자기하니 좋아요 근데 중반부 갈수록 반복적인 루틴 어인스토리 진입부터 초심잃은 동태눈 되기,True,15,반복적인 루틴 어인스토리 진입부터 초심잃은 동태눈 되기,"[반복, 적, 인, 루틴, 어인, 스토리, 진입, 부터, 초심, 잃은, 동태, 눈, 되기]"
6,할인 기다리다가 지쳐서 그냥 원가주고 샀는데 원가 주고 살만한 게임이네요 스토리도 꽤나 재밌고 일단 물고기 잡는게 재밌어서 시간 가는 줄 모르겠어요,True,14,살만한 게임이네요 스토리도 꽤나 재밌고 일단 물고기 잡는게 재밌어서,"[살, 만, 한, 게임, 이네, 요, 스토리, 도, 꽤, 나, 재밌고, 일단, 물고기, 잡는게, 재밌어서]"
10,어릴적 초창기 바람의 나라 메이플스토리가 돌아왔다,True,9,바람의 나라 메이플스토리가 돌아왔다,"[바람, 의, 나라, 메이플스토리, 가, 돌아왔다]"
8,얼리인데도 정가로 사기에 아깝지 않아요 게임 할 때 천천히 전부 맛보는편이라 플탐 생각보다 많이 나와서 당황 현재 스토리나 퀘스트는 다 끝냈고 초밥이랑 무기 다 만렙으로 만들어놓고 싶었으나 나중에 너무 쉬워질까봐 일단 킵해놨어요 빨리 정식버전 나왔으면 좋겠어요,True,7,나와서 당황 현재 스토리나 퀘스트는 다 끝냈고 초밥이랑 무기 다 만렙,"[나와서, 당황, 현재, 스토리, 나, 퀘스트, 는, 다, 끝냈고, 초밥, 이랑, 무기, 다, 만렙]"


### 부정적 리뷰

In [ ]:
# 부정적 리뷰만 추림 (12개)
opinion_story_minus = opinion_story[opinion_story[1] != True]
# 결과를 새로운 열에 저장
opinion_story_minus['부정'] = opinion_story_minus[0].apply(remove_before_story)
# 부정적 리뷰 키워드 분석 진행
tokenizer = Okt()
opinion_story_minus['부정'] = opinion_story_minus['부정'].apply(lambda x: ', '.join(x))
opinion_story_minus['리뷰 분석 형태소'] = opinion_story_minus['부정'].apply(tokenizer.morphs)
# 추출한 형태소 키워드 분석 진행
opinion_story_minus_mor = np.hstack(opinion_story_minus['리뷰 분석 형태소'].values)
# 단어의 빈도수를 계산
opinion_story_minus_mor = Counter(opinion_story_minus_mor)
opinion_story_minus_mor = Counter({x : opinion_story_minus_mor[x] for x in opinion_story_minus_mor if x not in stop_words})
opinion_story_minus_mor = sorted(opinion_story_minus_mor.items(), key=lambda x:x[1], reverse=True)
# 2번 이상 언급된 키워드만 출력
opinion_story_minus_mor = {word : count for word, count in opinion_story_minus_mor if count>= 2}
opinion_story_minus_mor
# 키워드 분석 결과 스토리가 흥미롭지 않다는 리뷰가 존재하는 것으로 예상 하지만 다른 리뷰는 예상 불가능

<ipython-input-169-f811aa40d023>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_story_minus['부정'] = opinion_story_minus[0].apply(remove_before_story)
<ipython-input-169-f811aa40d023>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinion_story_minus['부정'] = opinion_story_minus['부정'].apply(lambda x: ', '.join(x))
<ipython-input-169-f811aa40d023>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

{'스토리': 15,
 '만': 3,
 '못': 3,
 '하고': 3,
 '하는': 2,
 '진행': 2,
 '같은데': 2,
 ',': 2,
 '흥미롭지': 2,
 '않은': 2}

In [ ]:
# 추천 상위 10개 확인
opinion_story_minus.sort_values(by=2, ascending=False).head(10)

# 기타 (키워드와 관련 없는 내용, 키워드가 아닌 다른 어구 때문에 부정적 리뷰로 분류) : 2 | 1+1
# 흥미롭지 않은 스토리에 대한 불만 : 1 | 3
# 스토리 분량에 대한 불만 및 아쉬움 : 4 | 7+2+2+1
# 스토리에 대한 불만 : 1 | 2
# 스토리 내 미니게임에 대한 불만 : 1 | 1
# 길을 알려주지 않는 것에 대한 아쉬움 : 1 | 3

,0,1,2,부정,리뷰 분석 형태소
12,처음 구매하고 시간정도는 잠수장비 업글하고 하면서 재미있게 했는데 얼리라서 소지금 ...,False,7,지금 상한도 있고 스토리도 금방 끝나서 좀 묵혀놨다가 반년정도 이따가,"[지금, 상한, 도, 있고, 스토리, 도, 금방, 끝나서, 좀, 묵혀놨다가, 반년,..."
29,뭔가 프로 부족한 게임 메인 컨텐츠인 물고기 잡기 회떠서 팔기 자체는 재밌는데 후반...,False,3,없어 지루해진다 스토리는 차라리 길 다 알려주던 초반은 진행이 빠르,"[없어, 지루해진다, 스토리, 는, 차라리, 길, 다, 알려주던, 초반, 은, 진행..."
42,전개 느리고 지루하다 바다속 모든 움직임이 굼뜨니 짜증이 난다 게임플레이 챕터 까지...,False,3,"요소가 너무 많다 스토리 흥미롭지 않은 스토리, 리 흥미롭지 않은 스토리","[요소, 가, 너무, 많다, 스토리, 흥미롭지, 않은, 스토리, ,, 리, 흥미롭지..."
14,언제부턴가 같은거만 반복하는 느낌 스토리도 짧아서 정식 출시 후에 구매하시기 바래요,False,2,만 반복하는 느낌 스토리도 짧아서 정식 출시 후에 구매하시기 바래요,"[만, 반복, 하는, 느낌, 스토리, 도, 짧아서, 정식, 출시, 후, 에, 구매,..."
34,얼리엑세스 기준 시간 클리어 가능한 볼륨 업데이트 생각이 없음 재미는 있지만 왜 벌...,False,2,게임 켜고 중반 스토리도 못 온것 같은 찝찝함,"[게임, 켜고, 중반, 스토리, 도, 못, 온것, 같은, 찝찝함]"
35,장점 극 초반 바닷속이라는 배경이 신박하고 초반엔 힐링되는 느낌 적당한 타이쿤 식 ...,False,2,"포장한 것 같은데 스토리 상 굳이 불필요한 오타쿠 인물이 게임에 너무, 자주 등장하...","[포장, 한, 것, 같은데, 스토리, 상, 굳이, 불필요한, 오타쿠, 인물, 이, ..."
33,로 플레이할시 차나 맥주를 따르는것이 불가능함 잔이 채워지지가 않음 아마 콘솔용으로...,False,1,생긴것같은데 하필 스토리에 필수 미니게임으로 넣어놔서 게임 진행이 막,"[생긴것, 같은데, 하필, 스토리, 에, 필수, 미니게임, 으로, 넣어, 놔서, 게..."
36,시간 가까히 해보니까 느낌점 한 일은 할만합니다 일넘어가면서 컨텐츠 전부 다 해보니...,False,1,니다 인어마을에서 스토리 멈추고 사냥장사 무한대로 하다보니까 더 이상,"[니다, 인어, 마을, 에서, 스토리, 멈추고, 사냥, 장사, 무한, 대로, 하다,..."
37,재밌긴한데 얼리액세스라도 원은 좀 쎄다 생각함 게임 가 노가다 폰게임을 바람의나라처...,False,1,이 많다면 몰라도 스토리나 가게 성장만 보면 굉장히 단순하고 콘텐츠가,"[이, 많다면, 몰라도, 스토리, 나, 가게, 성장, 만, 보면, 굉장히, 단순하고..."
40,재미는있으나 역시 넥슨게임 넥슨타이틀을 벗어나지 못한 게임 같다 양산형게임들 보단 ...,False,1,서 풍경보고 특히 스토리에 집중하고 그러다 점점 빠져드는점이 매력인데,"[서, 풍경, 보고, 특히, 스토리, 에, 집중, 하고, 그러다, 점점, 빠져드는,..."
